In [36]:
import utils
import copy


# Reprezentarea problemei



In [37]:
class Teacher:
    def __init__(self, name, teacher_info):
        self.name = name
        self.preferred_days = set()
        self.preferred_intervals = set()
        self.courses = set()
        self.__days = set(['Luni', 'Marti', 'Miercuri', 'Joi', 'Vineri'])
        self.interval_count = 0

        for constraint in teacher_info['Constrangeri']:
            # Skip negative constraints, if a day/interval is not in the
            # positive constraint list, it is a negative constraint by default
            if constraint[0] == '!':
                continue

            if constraint in self.__days:
                self.preferred_days.add(constraint)
            else:
                two_hour_intervals = self.__get_two_hour_intervals_from_interval(constraint)
                self.preferred_intervals.update(two_hour_intervals)

        self.courses.update(teacher_info[utils.MATERII])

    def __str__(self):
        return f"{self.name} teaches {self.courses} and prefers {self.preferred_days} and {self.preferred_intervals}"

    def __get_two_hour_intervals_from_interval(self, interval):
        # Evaluate interval as a tuple
        start, end = eval(interval.replace('-', ','))

        result = []

        while start != end:
            result.append((start, start + 2))
            start += 2

        return result

class Course:
    def __init__(self, name, student_count):
        self.name = name
        self.student_count = student_count
        self.students_not_allocated = student_count
        self.teachers = set()
        self.classrooms = set()

    def add_teacher(self, teacher):
        self.teachers.add(teacher)

    def add_classroom(self, classroom):
        self.classrooms.add(classroom)

    def __str__(self):
        return f"{self.name} with {self.student_count} students, teachers {(list(map(lambda t: t.name, self.teachers)))} and classrooms {(list(map(lambda c: c.name, self.classrooms)))}"

class Classroom:
    def __init__(self, name, capacity, courses):
        self.name = name
        self.capacity = capacity
        self.courses = courses

    def __str__(self):
        return f"{self.name} with capacity {self.capacity} and courses {self.courses}"

In [38]:
def parse_yaml(yaml_path):
    yaml_dict = utils.read_yaml_file(yaml_path)
    courses_yaml = yaml_dict[utils.MATERII]
    classrooms_yaml = yaml_dict[utils.SALI]
    teachers_yaml = yaml_dict[utils.PROFESORI]

    teachers = dict()
    courses = dict()
    courses_not_allocated = dict()
    classrooms = dict()

    # Initialize courses
    for course_name, student_count in courses_yaml.items():
        course = Course(course_name, student_count)
        courses[course_name] = course
        courses_not_allocated[course_name] = student_count

    # Parse classroom info
    for classroom_name, classroom_info in classrooms_yaml.items():
        classroom = Classroom(classroom_name, classroom_info['Capacitate'], classroom_info[utils.MATERII])
        classrooms[classroom_name] = classroom
        for course in classroom_info[utils.MATERII]:
            courses[course].add_classroom(classroom)

    # Parse teacher info
    for teacher_name, teacher_info in teachers_yaml.items():
        teacher = Teacher(teacher_name, teacher_info)
        teachers[teacher_name] = teacher
        for course in teacher_info[utils.MATERII]:
            courses[course].add_teacher(teacher)

    return teachers, courses, courses_not_allocated, classrooms

teachers, courses, courses_not_allocated, classrooms = None, None, None, None

In [39]:
class State:
    """ Class that represents a partial timetable allocation """
    def __init__(self, input_yaml_path = None):
        self.timetable = dict()
        self.courses_not_allocated = dict()
        self.conflict_count = 0

        if input_yaml_path is not None:
            self.__parse_yaml_dict(utils.read_yaml_file(input_yaml_path))
            self.courses_not_allocated = copy.deepcopy(courses_not_allocated)

    def __str__(self):
        return utils.pretty_print_timetable(self.timetable)

    def __parse_yaml_dict(self, yaml_dict):
        # Initialize timetable with empty classrooms
        self.timetable.update({
            day: {
                eval(interval): {
                    classroom: None
                    for classroom in yaml_dict[utils.SALI]
                }
                for interval in yaml_dict[utils.INTERVALE]
            }
            for day in yaml_dict[utils.ZILE]
        })

    def is_final(self):
        for course_name in self.courses_not_allocated:
            if self.courses_not_allocated[course_name] > 0:
                return False

        return True

    def clone(self):
        state = State()
        state.timetable = copy.deepcopy(self.timetable)
        state.courses_not_allocated = copy.deepcopy(self.courses_not_allocated)
        state.conflict_count = self.conflict_count

        return state

    def fill_slot(self, day, interval, classroom, teacher_name, course_name):
        new_state = self.clone()
        new_state.timetable[day][interval][classroom] = (teacher_name, course_name)

        # Update the conflict count
        new_state.update_conflict_count()

        # Update the course not allocated count
        new_state.courses_not_allocated[course_name] -= classrooms[classroom].capacity

        return new_state

    def replace_slot(self, day, interval, classroom, teacher_name, course_name):
        new_state = self.clone()
        new_state.timetable[day][interval][classroom] = (teacher_name, course_name)

        # Update the conflict count
        new_state.update_conflict_count()

        # Update the course not allocated count
        new_state.courses_not_allocated[course_name] -= classrooms[classroom].capacity
        new_state.courses_not_allocated[self.timetable[day][interval][classroom][1]] += classrooms[classroom].capacity

        return new_state

    def get_next_states(self, course_name):
        """ Returns a generator of all possible states that can be reached from the current state """
        return (self.fill_slot(day, interval, classroom, teacher_name, course_name)
            for day in self.timetable
            for interval in self.timetable[day]
            for classroom in classrooms
            for teacher_name, teacher in teachers.items()
            if self.timetable[day][interval][classroom] is None and # slot not already filled
            course_name in teacher.courses and # teacher can teach the course
            course_name in classrooms[classroom].courses and # course can be held in the classroom
            self.courses_not_allocated[course_name] != 0 and # course has not been fully allocated
            self.get_teacher_interval_count(teacher_name) < 7 and # teacher has not reached the maximum number of intervals
            self.teacher_available_in_interval(teacher_name, interval, day)
        )

    def get_next_states_with_replacements(self, course_name):
        return (self.replace_slot(day, interval, classroom, teacher_name, course_name)
            for day in self.timetable
            for interval in self.timetable[day]
            for classroom in classrooms
            for teacher_name in teachers
            if self.timetable[day][interval][classroom] is not None and # slot already filled
            course_name in teachers[teacher_name].courses and # teacher can teach the course
            course_name in classrooms[classroom].courses and # course can be held in the classroom
            self.courses_not_allocated[course_name] != 0 and # course has not been fully allocated
            self.get_teacher_interval_count(teacher_name) < 7 and # teacher has not reached the maximum number of intervals
            self.teacher_available_in_interval(teacher_name, interval, day) and
            (self.timetable[day][interval][classroom][1] != course_name or
            self.timetable[day][interval][classroom][0] != teacher_name)
        )

    def eval(self):
        total = 0

        for day in self.timetable:
            for interval in self.timetable[day]:
                for classroom in self.timetable[day][interval]:
                    if self.timetable[day][interval][classroom] is not None:
                        teacher_name, _ = self.timetable[day][interval][classroom]

                        total += 100 / len(teachers[teacher_name].courses)

        total -= self.conflict_count * 100

        for course in self.courses_not_allocated:
            if self.courses_not_allocated[course] == 0:
                total += 25

        if self.is_final():
            total += 1000

        return total

    def teacher_available_in_interval(self, teacher_name, interval, day):
        for classroom in classrooms:
            timetable_entry = self.timetable[day][interval][classroom]

            if timetable_entry is not None and timetable_entry[0] == teacher_name:
                return False

        return True

    def get_teacher_interval_count(self, teacher_name):
        interval_count = 0
        for day in self.timetable:
            for interval in self.timetable[day]:
                for classroom in self.timetable[day][interval]:
                    if self.timetable[day][interval][classroom] is not None and self.timetable[day][interval][classroom][0] == teacher_name:
                        interval_count += 1
        return interval_count

    def update_conflict_count(self):
        self.conflict_count = 0

        for day in self.timetable:
            for interval in self.timetable[day]:
                for classroom in self.timetable[day][interval]:
                    if self.timetable[day][interval][classroom] is not None:
                        teacher_name, _ = self.timetable[day][interval][classroom]
                        teacher = teachers[teacher_name]

                        if day not in teacher.preferred_days:
                            self.conflict_count += 1

                        if interval not in teacher.preferred_intervals:
                            self.conflict_count += 1

    def get_least_flexible_course(self):
        courses_not_allocated = {course: self.courses_not_allocated[course] for course in self.courses_not_allocated if self.courses_not_allocated[course] > 0}
        course_name = min(courses_not_allocated, key=lambda course: courses[course].student_count)

        return course_name


In [40]:
def hill_climbing(initial: State, max_iters: int = 1000):
    iters, states = 0, 0
    state = initial.clone()

    while iters < max_iters:
        iters += 1

        if state.is_final(): # and state.conflict_count == 0:
            if state.conflict_count == 0:
                break
            else:
                break

        least_flexible_course = state.get_least_flexible_course()

        next_states = list(state.get_next_states(least_flexible_course))
        states += len(next_states)

        if len(next_states) == 0:
            next_states = list(state.get_next_states_with_replacements(least_flexible_course))
            states += len(next_states)

        if len(next_states) == 0:
            break

        new_state = max(next_states, key=lambda s: s.eval())

        if new_state.eval() < state.eval():
            break

        state = new_state

    return state.is_final(), iters, states, state


In [41]:
def run_test(test_path, overwrite_ref = False):
    global teachers, courses, courses_not_allocated, classrooms
    yaml_path = f"inputs/{test_path}.yaml"
    teachers, courses, courses_not_allocated, classrooms = parse_yaml(yaml_path)
    state = State(yaml_path)

    is_final, iters, states, final_state = hill_climbing(state)

    print(is_final)
    print(f"{final_state.conflict_count} conflicts")
    print(f"{iters} iterations")
    print(f"{states} states")

    print(utils.pretty_print_timetable(final_state.timetable))

    if overwrite_ref:
        # write to file
        with open(f"outputs/{test_path}.txt", "w") as f:
            f.write(utils.pretty_print_timetable(final_state.timetable))

In [42]:
run_test("dummy")

True
0 conflicts
12 iterations
204 states
|           Interval           |             Luni             |             Marti            |           Miercuri           |              Joi             |            Vineri            |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            8 - 10            |      MS : (EG324 - RG)       |      MS : (EG324 - CD)       |      MS : (EG324 - RG)       |
|                              |      DS : (EG390 - EG)       |      EG390 - goala           |      DS : (EG390 - EG)       |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            10 - 12           |      IA : (EG324 - PF)       |      EG324 - goala           |      IA : (EG324 - PF)       |
|               

In [43]:
run_test("orar_mic_exact")

True
0 conflicts
37 iterations
7053 states
|           Interval           |             Luni             |             Marti            |           Miercuri           |              Joi             |            Vineri            |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            8 - 10            |      ED010 - goala           |      ED010 - goala           |      ED010 - goala           |      ED010 - goala           |      ED010 - goala           |
|                              |      PL : (ED020 - AM)       |      PL : (ED020 - AM)       |      PCom : (ED020 - IS)     |      PCom : (ED020 - IS)     |      ED020 - goala           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            10 -

In [44]:
run_test("orar_mediu_relaxat")

True
0 conflicts
75 iterations
41333 states
|           Interval           |             Luni             |             Marti            |           Miercuri           |              Joi             |            Vineri            |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            8 - 10            |      MS : (ED038 - CA)       |      MS : (ED038 - CA)       |      SOC : (ED038 - RA)      |      PL : (ED038 - RE)       |      AA : (ED038 - EV)       |
|                              |      MS : (ED041 - IC)       |      MS : (ED041 - IC)       |      SOC : (ED041 - IG)      |      AA : (ED041 - EV)       |      ED041 - goala           |
|                              |      MS : (ED069 - PD)       |      MS : (ED069 - IG)       |      SOC : (ED069 - MA)      |      SOC : (ED069 - RA)      |      ED069 - goala           |
|               

In [45]:
run_test("orar_mare_relaxat")

True
0 conflicts
98 iterations
67573 states
|           Interval           |             Luni             |             Marti            |           Miercuri           |              Joi             |            Vineri            |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            8 - 10            |      MS : (ED090 - RA)       |      MS : (ED090 - RA)       |      ASC : (ED090 - EI)      |      MS : (ED090 - RA)       |      PL : (ED090 - AD)       |
|                              |      PCom : (ED091 - CC)     |      PCom : (ED091 - CC)     |      PCom : (ED091 - MA)     |      PCom : (ED091 - MA)     |      ASC : (ED091 - IG)      |
|                              |      EG346 - goala           |      EG346 - goala           |      EG346 - goala           |      ASC : (EG346 - EA)      |      EG346 - goala           |
|               

In [46]:
run_test("orar_constrans_incalcat", True)

True
5 conflicts
58 iterations
14958 states
|           Interval           |             Luni             |             Marti            |           Miercuri           |              Joi             |            Vineri            |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            8 - 10            |      PCom : (EC109 - ME)     |      PCom : (EC109 - IV)     |      DS : (EC109 - DD)       |      PM : (EC109 - CP)       |      SO : (EC109 - CA)       |
|                              |      PM : (ED043 - CP)       |      PM : (ED043 - CA)       |      SO : (ED043 - IV)       |      PM : (ED043 - VD)       |      SO : (ED043 - CP2)      |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            10 